## ใน Exercise นี้ เราจะวน loop ดึงข้อมูลทุกหน้าของบ้านเดี่ยวในกรุงเทพ จาก link ที่เราเก็บเป็น pickle เอาไว้ครับ

In [42]:
import urllib
from bs4 import BeautifulSoup
import pickle as pk
import openpyxl
import time
import numpy as np

In [43]:
with open('HouseLinkList.pkl','rb') as f:  # Python 3: open(..., 'rb')
    (HouseLinkList) = pk.load(f)

In [44]:
len(HouseLinkList)

1211

In [45]:
def func_Scrape_GenInfo(QuotePage):
    print(QuotePage)
    Page = urllib.request.urlopen(QuotePage)
    Soup = BeautifulSoup(Page, 'html.parser')

    Name  = Soup.find('div', attrs={'class': 'alt'}).text
    print(Name)
    
    PriceMin = Soup.find('div', attrs={'class': 'price'}).text
#    print(PriceMin)

# 1. Sales Information header section

    InfoMainTag = Soup.find('ul', attrs={'class': 'main-info-list'})
    InfoTagObj  = InfoMainTag.findAll('div', attrs={'class': 'value'})
    InfoCatTagObj  = InfoMainTag.findAll('div', attrs={'class': 'title'})
    
    print('---Info---')
    
    Location = ''
    Status   = ''
    Type     = ''
    Segment  = ''
    Build    = ''
    Developer= ''
    
    for k in range(0,len(InfoCatTagObj)):
        if InfoCatTagObj[k].text == 'Address':              Location = InfoTagObj[k].text
        if InfoCatTagObj[k].text == 'Status':               Status = InfoTagObj[k].text
        if InfoCatTagObj[k].text == 'Property Type':        Type = InfoTagObj[k].text
        if InfoCatTagObj[k].text == 'Segment':              Segment = InfoTagObj[k].text
        if InfoCatTagObj[k].text == 'Project Start Date':   Build = InfoTagObj[k].text
        if InfoCatTagObj[k].text == 'Developed By':         Developer = InfoTagObj[k].text
        
# 2. Basic Information section
    print('---Basic---')
    BasicMainTag = Soup.find('ul', attrs={'class': 'basic-info-list'})
    
    try:
        AreaTag = BasicMainTag.find('li', attrs={'class': 'basic-info-item area-total'})
        Area = AreaTag.text.strip().split('\n')[1]
        print('Area = ' +  Area)
    except AttributeError :
        Area = ''
        
        
    try:        
        NTypeTag = BasicMainTag.find('li', attrs={'class': 'basic-info-item num-unit-type'})
        NType = NTypeTag.text.strip().split('\n')[1]
        print('NType = ' + NType)
    except AttributeError :
        NType = ''

    try:          
        NUnitTag = BasicMainTag.find('li', attrs={'class': 'basic-info-item num-unit'})
        NUnit = NUnitTag.text.strip().split('\n')[1]    
        print('NUnit = ' + NUnit)
    except AttributeError :
        NUnit = ''        

    try:    
        NFloorTag = BasicMainTag.find('li', attrs={'class': 'basic-info-item num-floor'})
        NFloor = NFloorTag.text.strip().split('\n')[1]
        print('NFloor = ' + NFloor)
    except AttributeError :
        NFloor = ''

    try:    
        NParkingTag = BasicMainTag.find('li', attrs={'class': 'basic-info-item num-parking'})
        NParking = NParkingTag.text.strip().split('\n')[1]
        print('NParking = ' + NParking)
    except AttributeError :
        NParking = ''        
        
    try:    
        RParkingTag = BasicMainTag.find('li', attrs={'class': 'basic-info-item ratio-parking'})
        RParking = RParkingTag.text.strip().split('\n')[1]
        print('RParking = ' + RParking)
    except AttributeError :
        RParking = ''        

    try:    
        NLiftTag = BasicMainTag.find('li', attrs={'class': 'basic-info-item num-lift'})
        NLift = NLiftTag.text.strip().split('\n')[1]
        print('NLift = ' + NLift)
    except AttributeError :
        NLift = ''                
        
    try:         
        PriceStartTag = BasicMainTag.find('li', attrs={'class': 'basic-info-item price-start'})
        PriceStart = PriceStartTag.text.strip().split('\n')[1].split()[0]    
        print('PriceStart = ' + PriceStart)
    except AttributeError :
        PriceStart = ''
        
    try:        
        PriceHighTag = BasicMainTag.find('li', attrs={'class': 'basic-info-item price-end'})
        PriceHigh = PriceHighTag.text.strip().split('\n')[1].split()[0]       
        print('PriceHigh = ' + PriceHigh)
    except AttributeError :
        PriceHigh = ''
         

# 3. Image
    ImageUrl = Soup.find('div', attrs = {'class':'entity-main-image'}).find('img').get('src')
        
# 4. Location
#    print('location scraping')
    LocationTag = Soup.find('living-score-widget-map-canvas')
    if LocationTag == None:
        LocationTag = Soup.find('baania-map-streetview-overlay')
    
    try:
        Latitude = LocationTag.get('latitude')
    except  AttributeError :
        Latitude = ''
    
    try:
        Longitude = LocationTag.get('longitude')
    except AttributeError :
        Longitude = ''
#    print(Latitude,Longitude)    
    
# 5. Project Progress
    try:
        ProgressSect=Soup.find('div', attrs={'class': 'overall col-sm-3 col-lg-4'})
        Progress=ProgressSect.find('div', attrs={'class': 'value'}).text
    except AttributeError :
        Progress=''
        
    condoinfo=(Name, PriceMin, Location, Status,  Type, Segment, Build, Developer,
             Area, NType, NUnit,NFloor,NParking,RParking, NLift, PriceStart, PriceHigh,
             ImageUrl, Latitude, Longitude,Progress)
    
    return condoinfo,Soup

In [46]:
#q='https://baania.com/en/project/%E0%B9%80%E0%B8%8B%E0%B9%87%E0%B8%99%E0%B8%97%E0%B8%A3%E0%B8%B4%E0%B8%84-%E0%B8%AB%E0%B9%89%E0%B8%A7%E0%B8%A2%E0%B8%82%E0%B8%A7%E0%B8%B2%E0%B8%87-%E0%B8%AA%E0%B9%80%E0%B8%95%E0%B8%8A%E0%B8%B1%E0%B9%88%E0%B8%99?s=search&pid=BKK0605&province=3781&district=4000&ptype=2'
def func_find_RoomResale(Soup):  
#    Page = urllib.request.urlopen(q)
#    Soup = BeautifulSoup(Page, 'html.parser')
        
    # find all rooms in this page    
    AllRoomsTag= Soup.find('ul', attrs= {'class': 'listing-list item-list'})
    AllRooms= AllRoomsTag.findAll('li', attrs= {'class': 'listing-row item-row pie-clearfix'})
    print('------ Number of resale rooms inside this page is = ' + str(len(AllRooms)))
    return AllRooms

#func_find_RoomResale(q)

In [47]:
# extract all info in the room
def func_Resale_info(AllRooms,i): 
    # extract data for each room
    try: Link = AllRooms[i].find('h4', attrs={'class': 'title'}).find('a').get('href') # ดึง href ของแต่ละ room
    except AttributeError : Link =''
        
    try: Title = AllRooms[i].find('h4', attrs={'class': 'title'}).text.strip().split('\n')[0]
    except AttributeError : Title =''
        
    try: Area = AllRooms[i].find('div', attrs={'class': 'usable-area'}).text.strip().split(' ', 2)[0]
    except AttributeError : Area =''
        
    try: Price = AllRooms[i].find('div', attrs={'class': 'price'}).text.strip().split('\n')[0].split()[0].replace(',', '')
    except AttributeError : Price =''
        
    try: Bed = AllRooms[i].find('div', attrs={'class': 'col column-bed'}).text.strip().split('\n')[0]
    except AttributeError : Bed =''
            
    try: Bath = AllRooms[i].find('div', attrs={'class': 'col column-bath'}).text.strip().split('\n')[0]
    except AttributeError : Bath =''
            
    try: Parking = AllRooms[i].find('div', attrs={'class': 'col column-size'}).text.strip().split('\n')[0]
    except AttributeError : Parking =''  
            
    roominfo= (Link,Title,Area,Price,Bed,Bath,Parking)
    
#    print(roominfo)
    
    return roominfo

In [48]:
#q='https://baania.com/en/project/%E0%B9%80%E0%B8%8B%E0%B9%87%E0%B8%99%E0%B8%97%E0%B8%A3%E0%B8%B4%E0%B8%84-%E0%B8%AB%E0%B9%89%E0%B8%A7%E0%B8%A2%E0%B8%82%E0%B8%A7%E0%B8%B2%E0%B8%87-%E0%B8%AA%E0%B9%80%E0%B8%95%E0%B8%8A%E0%B8%B1%E0%B9%88%E0%B8%99?s=search&pid=BKK0605&province=3781&district=4000&ptype=2'
def func_find_RoomRent(Soup):  
#    Page = urllib.request.urlopen(q)
#    Soup = BeautifulSoup(Page, 'html.parser')
        
    # find all rooms in this page    
    AllRoomsTag= Soup.find('ul', attrs= {'class': 'rent-list item-list'})
    AllRooms= AllRoomsTag.findAll('li', attrs= {'class': 'rent-row item-row pie-clearfix'})
    print('------ Number of rental rooms inside this page is = ' + str(len(AllRooms)))
    return AllRooms

#func_find_RoomResale(q)

In [49]:
# extract all info in the room
def func_Rent_info(AllRooms,i): 
    # extract data for each room
    try: Link = AllRooms[i].find('h4', attrs={'class': 'title'}).find('a').get('href') # ดึง href ของแต่ละ room
    except AttributeError : Link =''
        
    try: Title = AllRooms[i].find('h4', attrs={'class': 'title'}).text.strip().split('\n')[0]
    except AttributeError : Title =''
        
    try: Area = AllRooms[i].find('div', attrs={'class': 'usable-area'}).text.strip().split(' ', 2)[0]
    except AttributeError : Area =''
        
    try: Price = AllRooms[i].find('div', attrs={'class': 'price'}).text.strip().split('\n')[0].split()[0].replace(',', '')
    except AttributeError : Price =''
        
    try: Bed = AllRooms[i].find('div', attrs={'class': 'col column-bed'}).text.strip().split('\n')[0]
    except AttributeError : Bed =''
            
    try: Bath = AllRooms[i].find('div', attrs={'class': 'col column-bath'}).text.strip().split('\n')[0]
    except AttributeError : Bath =''
            
    try: Parking = AllRooms[i].find('div', attrs={'class': 'col column-size'}).text.strip().split('\n')[0]
    except AttributeError : Parking =''  
            
    roominfo= (Link,Title,Area,Price,Bed,Bath,Parking)
    
#    print(roominfo)
    
    return roominfo

In [50]:
# Make database header (first row of excel database file)
    
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Housing_info"
dest_filename = 'Housing_info.xlsx'

current_col=0
allinfo=['Name', 'PriceMin', 'Location', 'Status','Type', 'Segment', 'Build', 'Developer',
         'CondoArea', 'NType', 'NUnit','NFloor','NParking','RParking', 'NLift', 'PriceStart', 'PriceHigh',
         'ImageUrl', 'Latitude', 'Longitude','Progress',
        'LinkS','TitleS','RoomAreaS','PriceS','BedS','BathS','ParkingS', # Resale Room info
        'LinkR','TitleR','RoomAreaR','PriceR','BedR','BathR','ParkingR'] # Rental Room info
for i in range(0,len(allinfo)):
    ws.cell(row=1,column=i+1).value = str(allinfo[i])
    current_col=current_col+1
    
wb.save(filename = dest_filename)


In [51]:
# Get info for the page
start_time = time.time()
start_row = 2
r=start_row

for c in range(989,len(HouseLinkList)): #len(HouseLinkList):
    print('----------------------------------Current Iteration is :' + str(c))
    condoinfo,Soup = func_Scrape_GenInfo(HouseLinkList[c])     
    
    try: # check if resale room is available                
        AllRooms=func_find_RoomResale(Soup)            
        for i in range(0,len(AllRooms)): # loop all rooms in condo
            roominfo = func_Resale_info(AllRooms,i)
            for i in range(0,len(condoinfo)):
                ws.cell(row=r,column=i+1).value = condoinfo[i]              
            for i in range(0,len(roominfo)):
                ws.cell(row=r,column=len(condoinfo)+i+1).value = roominfo[i]                       
            r+=1          
        try: # check if rental room is available                
            AllRooms=func_find_RoomRent(Soup)            
            for i in range(0,len(AllRooms)): # loop all rooms in condo             
                roominfo = func_Rent_info(AllRooms,i)
                for i in range(0,len(condoinfo)):
                    ws.cell(row=r,column=i+1).value = condoinfo[i]              
                for i in range(0,len(roominfo)):
                    ws.cell(row=r,column=len(roominfo)+len(condoinfo)+i+1).value = roominfo[i]                       
                r+=1     
        except AttributeError: 
            pass       

        
    except AttributeError: # if no resale for renting
        try: # check if rental room is available                
            AllRooms=func_find_RoomRent(Soup)            
            for i in range(0,len(AllRooms)): # loop all rooms in condo                
                roominfo = func_Rent_info(AllRooms,i)
                for i in range(0,len(condoinfo)):
                    ws.cell(row=r,column=i+1).value = condoinfo[i]              
                for i in range(0,len(roominfo)):
                    ws.cell(row=r,column=len(roominfo)+len(condoinfo)+i+1).value = roominfo[i]                       
                r+=1 
                
        except AttributeError: # add only condo info            
            for i in range(0,len(condoinfo)): # add condo info for each room
                ws.cell(row=r,column=i+1).value = condoinfo[i]
            r+=1 
        
    wb.save(filename = dest_filename)         
    print('Completed a Condo')
    print('[Chris]: Data Complete time = ' + str(time.time() - start_time) + ' seconds')
    time.sleep(np.random.random_sample())    # to slow down process after load each page    
print('---------------- Scraping Completed ----------------')    


    # To scrape picture enable this code
    # urllib.request.urlretrieve(ImageUrl, 'ID_' + str(i+1) + '.jpg') # to extract picture





----------------------------------Current Iteration is :989
https://baania.com/en/project/%E0%B8%A5%E0%B8%B5-%E0%B9%80%E0%B8%9E%E0%B8%A5%E0%B8%AA-%E0%B8%9E%E0%B8%B1%E0%B8%92%E0%B8%99%E0%B8%B2%E0%B8%81%E0%B8%B2%E0%B8%A3-%E0%B8%84%E0%B8%AD%E0%B8%99%E0%B9%82%E0%B8%94?s=search&pid=BKK1066&province=3781&district=3901&ptype=2
Lee Place Phattanakarn Condo
---Info---
---Basic---
NType = 4
NFloor = 8
Completed a Condo
[Chris]: Data Complete time = 1.9320595264434814 seconds
----------------------------------Current Iteration is :990
https://baania.com/en/project/%E0%B9%80%E0%B8%AD%E0%B8%AA%E0%B9%80%E0%B8%9E%E0%B8%99-%E0%B8%A5%E0%B8%B2%E0%B8%8B%E0%B8%B2%E0%B8%A5?s=search&pid=BKK0525&province=3781&district=3877&ptype=2
Aspen Lasalle
---Info---
---Basic---
Area = 3 Rai 3 Ngan 78.1 Sq.Wah
NType = 4
NUnit = 425
NFloor = 8
NParking = 140
RParking = 35 %
NLift = 2
PriceStart = 1.59
PriceHigh = 2.45
Completed a Condo
[Chris]: Data Complete time = 3.702277421951294 seconds
------------------------------

Supalai Park Srinakarin
---Info---
---Basic---
Area = 13 Rai 2 Ngan 13 Sq.Wah
NType = 3
NUnit = 1405
NFloor = 25
NParking = 1000
PriceStart = 1.39
------ Number of rental rooms inside this page is = 3
Completed a Condo
[Chris]: Data Complete time = 31.385650157928467 seconds
----------------------------------Current Iteration is :1004
https://baania.com/en/project/%E0%B9%84%E0%B8%AD%E0%B8%94%E0%B8%B5%E0%B9%82%E0%B8%AD-%E0%B9%82%E0%B8%A1%E0%B8%9A%E0%B8%B4-%E0%B8%9E%E0%B8%8D%E0%B8%B2%E0%B9%84%E0%B8%97?s=search&pid=BKK2670&province=3781&district=3942&ptype=2
Ideo Mobi Phayathai
---Info---
---Basic---
Area = 1 Rai 2 Ngan 77 Sq.Wah
NType = 4
NUnit = 316
NFloor = 24
NParking = 129
RParking = 40 %
NLift = 2
PriceStart = 4.00
------ Number of resale rooms inside this page is = 1
------ Number of rental rooms inside this page is = 1
Completed a Condo
[Chris]: Data Complete time = 33.51652193069458 seconds
----------------------------------Current Iteration is :1005
https://baania.com/en/project

Supalai Park Ratchapruek-Phetkasem
---Info---
---Basic---
Area = 6 Rai 2 Ngan 32 Sq.Wah
NType = 3
NUnit = 835
NFloor = 35
NParking = 340
RParking = 40 %
NLift = 4
PriceStart = 2.50
------ Number of resale rooms inside this page is = 2
------ Number of rental rooms inside this page is = 1
Completed a Condo
[Chris]: Data Complete time = 59.71755266189575 seconds
----------------------------------Current Iteration is :1018
https://baania.com/en/project/%E0%B8%9E%E0%B8%A5%E0%B8%B1%E0%B8%AA-38-%E0%B8%AE%E0%B8%B4%E0%B8%9B?s=search&pid=BKK3015&province=3781&district=3782&ptype=2
Plus 38 Hip
---Info---
---Basic---
NType = 3
NUnit = 112
NFloor = 8
PriceStart = 3.30
------ Number of resale rooms inside this page is = 11
------ Number of rental rooms inside this page is = 15
Completed a Condo
[Chris]: Data Complete time = 61.79306125640869 seconds
----------------------------------Current Iteration is :1019
https://baania.com/en/project/%E0%B8%98%E0%B8%99%E0%B8%9E%E0%B8%A5-%E0%B8%84%E0%B8%AD%E0%B

----------------------------------Current Iteration is :1032
https://baania.com/en/project/%E0%B8%AD%E0%B8%B8%E0%B8%94%E0%B8%A1%E0%B8%AA%E0%B8%B8%E0%B8%82%E0%B8%AD%E0%B8%B2%E0%B8%84%E0%B8%B2%E0%B8%A3%E0%B8%8A%E0%B8%B8%E0%B8%94?s=search&pid=BKK1082&province=3781&district=3901&ptype=2
Udomsuk Arkarnchud
---Info---
---Basic---
NType = 1
NUnit = 51
NFloor = 4
PriceStart = 8.50
Completed a Condo
[Chris]: Data Complete time = 95.99578499794006 seconds
----------------------------------Current Iteration is :1033
https://baania.com/en/project/%E0%B9%81%E0%B8%97%E0%B8%A1%E0%B8%A1%E0%B8%B2%E0%B8%A3%E0%B8%B5%E0%B8%99-%E0%B8%97%E0%B8%A3%E0%B8%B5-%E0%B9%80%E0%B8%A3%E0%B8%AA%E0%B8%8B%E0%B8%B4%E0%B9%80%E0%B8%94%E0%B8%99%E0%B8%8B%E0%B9%8C?s=search&pid=BKK2741&province=3781&district=3800&ptype=2
Tammareen Tree Resident
---Info---
---Basic---
NType = 2
NUnit = 72
NFloor = 8
NParking = 45
RParking = 60 %
NLift = 1
PriceStart = 2.11
PriceHigh = 3.06
Completed a Condo
[Chris]: Data Complete time = 99.68314

Completed a Condo
[Chris]: Data Complete time = 131.18667912483215 seconds
----------------------------------Current Iteration is :1047
https://baania.com/en/project/%E0%B8%AD%E0%B8%AD%E0%B8%A3%E0%B8%B4%E0%B8%88%E0%B8%B4%E0%B8%99%E0%B8%AA%E0%B9%8C-%E0%B8%9E%E0%B8%A3%E0%B8%B0%E0%B8%A3%E0%B8%B2%E0%B8%A1-2?s=search&pid=BKK2979&province=3781&district=3859&ptype=2
Origins Rama II
---Info---
---Basic---
Area = 2 Rai 3 Ngan 75 Sq.Wah
NType = 2
NUnit = 471
NFloor = 8
NParking = 142
RParking = 30 %
NLift = 2
PriceStart = 0.80
Completed a Condo
[Chris]: Data Complete time = 132.76913189888 seconds
----------------------------------Current Iteration is :1048
https://baania.com/en/project/%E0%B8%AA%E0%B8%B5%E0%B8%A5%E0%B8%A1-%E0%B8%84%E0%B8%AD%E0%B8%99%E0%B9%82%E0%B8%94%E0%B8%A1%E0%B8%B4%E0%B9%80%E0%B8%99%E0%B8%B5%E0%B8%A2%E0%B8%A1?s=search&pid=BKK2443&province=3781&district=3886&ptype=2
Silom Condominuim
---Info---
---Basic---
NType = 1
NUnit = 38
NFloor = 12
PriceStart = 10.32
Completed a Condo


...

Palm Suite Complex
---Info---
---Basic---
NType = 1
NUnit = 280
NFloor = 9
Completed a Condo
[Chris]: Data Complete time = 376.5699589252472 seconds
----------------------------------Current Iteration is :1152
https://baania.com/en/project/%E0%B8%AA%E0%B8%B4%E0%B8%99%E0%B9%80%E0%B8%A8%E0%B8%A3%E0%B8%A9%E0%B8%90%E0%B8%B5%E0%B8%84%E0%B8%AD%E0%B8%A1%E0%B9%80%E0%B8%9E%E0%B8%A5%E0%B9%87%E0%B8%81%E0%B8%8B%E0%B9%8C?s=search&pid=BKK2291&province=3781&district=3856&ptype=2
Sin Setthi Complex
---Info---
---Basic---
NType = 1
NFloor = 8
PriceStart = 0.55
Completed a Condo
[Chris]: Data Complete time = 379.26733779907227 seconds
----------------------------------Current Iteration is :1153
https://baania.com/en/project/%E0%B8%A2%E0%B8%B9-%E0%B8%94%E0%B8%B5%E0%B9%84%E0%B8%A5%E0%B8%97%E0%B9%8C-%E0%B9%80%E0%B8%A3%E0%B8%AA%E0%B8%8B%E0%B8%B4%E0%B9%80%E0%B8%94%E0%B9%89%E0%B8%99%E0%B8%8B%E0%B9%8C-%E0%B8%9E%E0%B8%B1%E0%B8%92%E0%B8%99%E0%B8%B2%E0%B8%81%E0%B8%B2%E0%B8%A3?s=search&pid=BKK6237&province=3781&di

The Room Sukhumvit 62
---Info---
---Basic---
Area = 5 Rai 3 Ngan 23 Sq.Wah
NType = 2
NUnit = 487
NFloor = 23
NParking = 343
RParking = 70 %
NLift = 6
PriceStart = 4.39
------ Number of resale rooms inside this page is = 19
------ Number of rental rooms inside this page is = 40
Completed a Condo
[Chris]: Data Complete time = 417.56060552597046 seconds
----------------------------------Current Iteration is :1168
https://baania.com/en/project/%E0%B9%80%E0%B8%94%E0%B8%AD%E0%B8%B0%E0%B8%97%E0%B8%A3%E0%B8%B1%E0%B8%AA%E0%B8%95%E0%B9%8C-%E0%B9%80%E0%B8%A3%E0%B8%AA%E0%B8%8B%E0%B8%B4%E0%B9%80%E0%B8%94%E0%B9%89%E0%B8%99%E0%B8%8B%E0%B9%8C-%E0%B8%9B%E0%B8%B4%E0%B9%88%E0%B8%99%E0%B9%80%E0%B8%81%E0%B8%A5%E0%B9%89%E0%B8%B2?s=search&pid=BKK2271&province=3781&district=3847&ptype=2
The Trust Residence Pinklao
---Info---
---Basic---
Area = 6 Rai 2 Ngan 38 Sq.Wah
NType = 1
NUnit = 1408
NFloor = 30
RParking = 35 %
NLift = 4
PriceStart = 2.20
------ Number of resale rooms inside this page is = 5
------ Numbe

----------------------------------Current Iteration is :1182
https://baania.com/en/project/%E0%B8%8B%E0%B8%B4%E0%B8%95%E0%B8%B5%E0%B9%89%E0%B8%A7%E0%B8%B4%E0%B8%A5%E0%B8%A5%E0%B9%88%E0%B8%B2?s=search&pid=BKK2284&province=3781&district=3856&ptype=2
city villa
---Info---
---Basic---
NType = 1
NUnit = 1748
NFloor = 9
NLift = 2
PriceStart = 0.70
------ Number of resale rooms inside this page is = 1
Completed a Condo
[Chris]: Data Complete time = 453.43594765663147 seconds
----------------------------------Current Iteration is :1183
https://baania.com/en/project/%E0%B9%81%E0%B8%9A%E0%B8%87%E0%B8%84%E0%B9%8C%E0%B8%84%E0%B8%AD%E0%B8%81-%E0%B9%80%E0%B8%9F%E2%80%99%E0%B8%A5%E0%B8%B4%E0%B8%8B-%E0%B8%AA%E0%B8%B2%E0%B8%97%E0%B8%A3-%E0%B8%95%E0%B8%B2%E0%B8%81%E0%B8%AA%E0%B8%B4%E0%B8%99?s=search&pid=BKK0108&province=3781&district=3786&ptype=2
Bangkok Feliz Sathorn-Taksin
---Info---
---Basic---
Area = 1 Rai 1 Ngan 61 Sq.Wah
NType = 4
NUnit = 200
NFloor = 8
NParking = 93
RParking = 40 %
NLift = 2
Pric

----------------------------------Current Iteration is :1196
https://baania.com/en/project/%E0%B9%80%E0%B8%8B%E0%B9%87%E0%B8%99%E0%B8%97%E0%B8%A3%E0%B8%B4%E0%B8%84-%E0%B8%8B%E0%B8%B5%E0%B8%99-%E0%B8%9E%E0%B8%AB%E0%B8%A5%E0%B9%82%E0%B8%A2%E0%B8%98%E0%B8%B4%E0%B8%99-9?s=search&pid=BKK1127&province=3781&district=3911&ptype=2
Centric Scene Phaholyothin 9
---Info---
---Basic---
Area = 1 Rai 2 Ngan
NType = 3
NUnit = 206
NFloor = 24
NParking = 223
PriceStart = 2.55
PriceHigh = 7.00
------ Number of resale rooms inside this page is = 1
------ Number of rental rooms inside this page is = 2
Completed a Condo
[Chris]: Data Complete time = 490.93986892700195 seconds
----------------------------------Current Iteration is :1197
https://baania.com/en/project/%E0%B8%97%E0%B8%A3%E0%B8%B5-%E0%B8%84%E0%B8%AD%E0%B8%99%E0%B9%82%E0%B8%94-%E0%B8%AA%E0%B8%B8%E0%B8%82%E0%B8%B8%E0%B8%A1%E0%B8%A7%E0%B8%B4%E0%B8%97-50?s=search&pid=BKK4058&province=3781&district=3782&ptype=2
Tree Condo Sukhumvit 50
---Info---
---B

---------------- Scraping Completed ----------------
